In [2]:
import pandas as pd

In [4]:
with open('eurlex.taxonomy', 'r') as f:
    lines = f.readlines()

hierarchy = {}
for line in lines:
    line = line.strip(' \n')
    line = line.split('\t')
    hierarchy[line[0]] = line[1:]

r_hiera = {}
for k, v in hierarchy.items():
    for i in v:
        if i not in r_hiera:
            r_hiera[i] = [k]
        else:
            print(k)
            r_hiera[i].append(k)
r_hiera

trade
agri-foodstuffs


{'politics': ['Root'],
 'international relations': ['Root'],
 'european union': ['Root'],
 'law': ['Root'],
 'economics': ['Root'],
 'trade': ['Root', 'trade'],
 'finance': ['Root'],
 'social questions': ['Root'],
 'education and communications': ['Root'],
 'science': ['Root'],
 'business and competition': ['Root'],
 'employment and working conditions': ['Root'],
 'transport': ['Root'],
 'environment': ['Root'],
 'agriculture, forestry and fisheries': ['Root'],
 'agri-foodstuffs': ['Root', 'agri-foodstuffs'],
 'production, technology and research': ['Root'],
 'energy': ['Root'],
 'industry': ['Root'],
 'geography': ['Root'],
 'international organisations': ['Root'],
 'political framework': ['politics'],
 'political party': ['politics'],
 'electoral procedure and voting': ['politics'],
 'parliament': ['politics'],
 'parliamentary proceedings': ['politics'],
 'politics and public safety': ['politics'],
 'executive power and public service': ['politics'],
 'international affairs': ['inter

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
a = torch.randint(0, 5, (3, 4))
b = torch.randint(0, 5, (2, 4))

print(a, b)

tensor([[3, 1, 1, 3],
        [0, 0, 0, 1],
        [4, 1, 3, 2]]) tensor([[0, 4, 3, 1],
        [2, 3, 2, 2]])


In [17]:
# convert a and b to dtype=torch.float32
a = a.float()
b = b.float()


In [18]:
b1 = torch.tensor([[0, 4, 3, 1]])
b1 = b1.float()
b1.shape

torch.Size([1, 4])

In [23]:
torch.mean((a[:, None, :] - b1) * (a[:, None, :]), dim=2).transpose(0, 1)

tensor([[2.5000, 0.0000, 3.7500]])

In [22]:
print(a[:, None, :])
torch.mean((a[:, None, :] - b) * (a[:, None, :]), dim=2).transpose(0, 1)

tensor([[[3., 1., 1., 3.]],

        [[0., 0., 0., 1.]],

        [[4., 1., 3., 2.]]])


tensor([[ 2.5000,  0.0000,  3.7500],
        [ 0.7500, -0.2500,  2.2500]])

In [32]:
tensor_example = [[4.46, 7.57, 7.58]]
# tensor_example = tensor_example
tensor_example = torch.tensor(tensor_example)

# plot the softmax output
import matplotlib.pyplot as plt
import numpy as np


# def plot_softmax(tensor_example, knn_T=10):
#     tensor_example = tensor_example / knn_T
#     softmax = torch.nn.Softmax(dim=1)
#     softmax_output = softmax(tensor_example)
#     print(softmax_output)
#     plt.plot(softmax_output.detach().numpy().flatten(), 'o')
#     # shot the x axis as discrete values
#     plt.xticks(np.arange(16))
#     # plot the y from 0 to 1
#     plt.ylim(0, 1)
#     plt.show()

# plot_softmax(tensor_example, 100)
torch.softmax(tensor_example, dim=-1)

tensor([[0.0217, 0.4867, 0.4916]])

In [5]:
tokenizer = AutoTokenizer.from_pretrained("gpt2-xl")
tokenizer(" positive")

{'input_ids': [3967], 'attention_mask': [1]}

In [2]:
'''
Choose sentiment from terrible or great.

Review: i would recommend big bad love only to winger fans who have missed her since 1995 's forget paris.
Sentiment: terrible
Review:  suspenseful enough for older kids but not . 
Sentiment: great

Review: the subtle strength of elling is that it never squandering touch with the reality of the grim situation . 
Sentiment:
'''
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-hf").to('cuda')

gpt_tokenizer = AutoTokenizer.from_pretrained("gpt2-xl")
gpt_model = AutoModelForCausalLM.from_pretrained("gpt2-xl").to('cuda')

# mistral_tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")
# mistral_model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-v0.1").to('cuda')

Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.37s/it]


In [11]:
gpt_tokenizer = AutoTokenizer.from_pretrained("gpt2-xl")
gpt_model = AutoModelForCausalLM.from_pretrained("gpt2-xl").to('cuda')

In [59]:
prompt = """Classify the sentiment of negative and positive.

Review: is a step down for director gary fleder . .
Sentiment: negative
Review: the director , tom dey , had spliced together bits and pieces of midnight run and 48 hours ( and , for that matter , shrek ) .
Sentiment: positive
Review: from two fatal ailments -- a dearth of vitality and a story that 's shapeless and uninflected .
Sentiment: negative
Review: results that are sometimes bracing .
Sentiment: positive
Review: plodding soap opera .
Sentiment: negative
Review: all-star salute .
Sentiment: positive
Review: fit all of pootie tang in between its punchlines .
Sentiment: negative
Review: award-winning .
Sentiment: positive
Review: deserve better . .
Sentiment: negative
Review: you actually buy into
Sentiment: positive
Review: of cliches that shoplifts shamelessly from farewell-to-innocence movies like the wanderers and a bronx tale without cribbing any of their intelligence .
Sentiment: negative
Review: real-life basis is , in fact , so interesting that no embellishment is
Sentiment: positive
Review: to insulting the intelligence of anyone who has n't been living under a rock
Sentiment: negative
Review: immensely ambitious
Sentiment: positive
Review: into the modern rut of narrative banality
Sentiment: negative
Review: user-friendly
Sentiment: positive
Review: I am so pretty . .
Sentiment:"""

tokenizer.padding_side = "left"
tokenizer.truncation_side = "left"
tokenizer.pad_token = tokenizer.eos_token
prompt_ids = tokenizer.encode(prompt, return_tensors="pt")
prompt_ids = prompt_ids.to('cuda')
# input_ids = tokenizer.encode(input, return_tensors="pt")

# input_ids = torch.cat([prompt_ids, input_ids[:, 1:]], dim=-1).to('cuda')

output = model(prompt_ids, return_dict=True).logits
print(output.shape)
output = output[:, -1, :]
tokenizer.decode(torch.argmax(output, dim=-1).squeeze().tolist())

torch.Size([1, 405, 32000])


'positive'

In [50]:
tokenizer("terrible")

{'input_ids': [1, 16403], 'attention_mask': [1, 1]}

In [24]:
prompt = """Classify the sentiment of positive and negative.
Review: i would recommend big bad love only to winger fans who have missed her since 1995 's forget paris . .
Sentiment: negative
Review: suspenseful enough for older kids but not .
Sentiment: positive
Review: another run-of-the-mill disney sequel intended for the home video market .
Sentiment: negative
Review: has never been smoother or more confident .
Sentiment: positive
Review: bad-movie .
Sentiment: negative
Review: sweetly .
Sentiment: positive
Review: like an extended dialogue exercise in retard 101 .
Sentiment: negative
Review: bouquet gives a performance that is masterly . .
Sentiment: positive"""

input1 = "Review: one of creepiest, scariest movies to come along in a long, long time, easily rivaling blair witch or the others. \nSentiment:"
input2 = "Review: good movie . \nSentiment:"

prompt_ids = gpt_tokenizer.encode(prompt, return_tensors="pt")
inputs1 = prompt + input1
inputs2 = prompt + input2

inputs = [inputs1, inputs2]
# inputs = [inputs1, inputs1]

gpt_tokenizer.pad_token = gpt_tokenizer.eos_token
input_ids = gpt_tokenizer.batch_encode_plus(inputs, return_tensors="pt", padding=True, truncation=True)['input_ids'].to('cuda')

attention_mask = input_ids.ne(gpt_tokenizer.pad_token_id).float().to('cuda')
output = gpt_model(input_ids, attention_mask=attention_mask, return_dict=True).logits

last_non_pad_indices = torch.ne(input_ids, gpt_tokenizer.pad_token_id).sum(-1) - 1
print(last_non_pad_indices)
output = output[range(output.shape[0]), last_non_pad_indices, :]

gpt_tokenizer.batch_decode(torch.argmax(output, dim=-1).squeeze().tolist())

tensor([193, 167], device='cuda:0')


[' negative', ' positive']

In [33]:
prompt = "Classify the sentiment of positive and negative.\n Review: i would recommend big bad love only to winger fans who have missed her since 1995 's forget paris.Sentiment: negative\nReview:  suspenseful enough for older kids but not . \nSentiment: positive\n"

input = "Review: the cd is not suitable for children . \nSentiment:"
# try with the mistral tokenizer and model

prompt_ids = mistral_tokenizer.encode(prompt, return_tensors="pt")
input_ids = mistral_tokenizer.encode(input, return_tensors="pt")

input_ids = torch.cat([prompt_ids, input_ids[:, 1:]], dim=-1).to('cuda')

# add 10 padding tokens to the end of the input
input_ids = torch.cat([input_ids, torch.ones((1, 10)).long().to('cuda')], dim=-1)

attention_mask = input_ids != 1

output = mistral_model(input_ids, attention_mask=attention_mask, return_dict=True).logits
print(output.shape)
output = output[:, len(input_ids[0]) - 11, :]
mistral_tokenizer.decode(torch.argmax(output, dim=-1).squeeze().tolist())

torch.Size([1, 90, 32000])


'negative'

In [22]:
mistral_tokenizer("great")

{'input_ids': [1, 1598], 'attention_mask': [1, 1]}

In [51]:
gpt_tokenizer(" terrible")

{'input_ids': [353, 5547], 'attention_mask': [1, 1]}